<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,91,jitters
1,93,shooting
2,187,dry mouth
3,189,dizzy
4,230,mania


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-3.test.csv",header=None)
test.head()

,0,1
0,13,insomniac
1,16,couldn't taste anything
2,61,spins
3,85,tremor
4,94,anxiety


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR_pubmed.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'for',
 'pain',
 'sodium',
 'drug',
 'by',
 'on',
 'patients',
 'as',
 'that',
 'is',
 'at',
 'or',
 'mg',
 'increased',
 'group',
 'from',
 'mood',
 'this',
 'study',
 'treatment',
 'after',
 'p',
 'an',
 'release',
 ',',
 'be',
 'me',
 'disorder',
 'effect',
 'not',
 'withdrawal',
 'abnormal',
 'i',
 'anxiety',
 'weight',
 'than',
 'decreased',
 'compared',
 'using',
 'effects',
 'are',
 'my',
 'symptoms',
 'inflammatory',
 'groups',
 'no',
 'both',
 'drugs',
 'two',
 'anti',
 'significantly',
 'acute',
 'potassium',
 'significant',
 'skin',
 'day',
 'between',
 'all',
 'disorders',
 'behavior',
 'used',
 'blood',
 'syndrome',
 'h',
 'dose',
 '(',
 ')',
 'voltaren',
 '-',
 'these',
 'time',
 'which',
 'had',
 'results',
 'it',
 'sleeplessness',
 'placebo',
 'efficacy',
 'diseases',
 'concentration',
 'days',
 'o

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.830500,4.232921,0.342593,17:30


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.020958,3.836071,0.377414,17:14


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.738058,3.793368,0.382011,17:15


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.686475,3.759337,0.385160,17:09


In [25]:
learn.save_encoder('TwADR_pubmed_fold3_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR_pubmed_fold3_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.217832,6.538230,0.058824,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.249514,5.148288,0.126050,00:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.744302,4.911015,0.184874,01:07


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.478743,4.554427,0.252101,01:07
1,5.234647,4.510120,0.226891,01:05


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.988869,4.236017,0.226891,01:08
1,4.758487,4.151838,0.226891,01:05


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.540300,3.985961,0.268908,01:02
1,4.390440,3.863149,0.336134,01:04


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.192036,3.674914,0.336134,01:09
1,4.029513,3.615359,0.344538,01:03


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.858003,3.509044,0.394958,01:06
1,3.698105,3.397321,0.378151,01:03


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.524100,3.393491,0.403361,01:02
1,3.444320,3.223316,0.378151,01:04
2,3.292889,3.184286,0.378151,01:05
3,3.105115,3.148206,0.369748,01:03


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.966757,3.153121,0.394958,01:08
1,2.893182,3.173349,0.394958,01:05
2,2.713536,3.090799,0.394958,01:08
3,2.548194,3.077898,0.403361,01:03


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.389088,3.169951,0.386555,01:02
1,2.338293,3.066400,0.403361,01:04
2,2.200472,3.074890,0.428571,01:12
3,2.033156,3.033796,0.453782,01:09


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.916799,3.004474,0.462185,01:07
1,1.867114,3.045572,0.445378,01:06
2,1.736668,3.074270,0.445378,01:06
3,1.606030,3.032726,0.462185,01:08


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.497773,3.010112,0.470588,01:07
1,1.467036,3.122963,0.453782,01:08
2,1.364610,3.086202,0.470588,01:04
3,1.244555,3.070336,0.470588,01:07


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.152172,3.072099,0.436975,01:08
1,1.152673,3.155343,0.453782,01:08
2,1.074122,3.131768,0.462185,01:07
3,0.990669,3.133755,0.445378,01:05


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.892796,3.192088,0.428571,01:04
1,0.932244,3.325774,0.453782,01:06
2,0.872955,3.382558,0.478992,01:06
3,0.791076,3.312671,0.478992,01:07


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.763300,3.236764,0.462185,01:08
1,0.715640,3.312828,0.462185,01:07


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.709949,3.435697,0.462185,01:03
1,0.677598,3.435889,0.470588,01:08


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.633214,3.535992,0.478992,01:07


In [44]:
# save the best model

learn.save_encoder('TwADR_pubmed_fold3')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.616367,3.631483,0.453782,01:08


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.568166,3.646473,0.428571,01:03


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.560923,3.632222,0.445378,01:09


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[2149, 1962, 1295, 1204, 978, 750, 109, 1610, 189, 1007, 233, 249, 1164, 2149, 1597, 2149, 1054, 1599, 2191, 393, 1787, 107, 1341, 1529, 2063, 232, 1373, 579, 629, 1483, 1204, 749, 749, 749, 792, 792, 792, 974, 579, 579, 579, 1308, 1942, 233, 1295, 908, 1099, 512, 974, 974, 1099, 232, 2147, 1669, 107, 1099, 1974, 1099, 1925, 1136, 1136, 1145, 749, 1204, 107, 1340, 1265, 2149, 189, 189, 2147, 1597, 1597, 1942, 1508, 749, 2074, 1484, 1787, 418, 452, 1545, 1099, 1545, 74, 1783, 1727, 1597, 1610, 1610, 1073, 824, 233, 1701, 2149, 1711, 1711, 1815, 1727, 1815, 2191, 1767, 1767, 1767, 1787, 13, 1787, 1787, 1787, 2150, 393, 393, 1787, 1815, 978, 1942, 825, 825, 792, 1915, 1099, 1936, 1936, 749, 1942, 518, 1767, 393, 13, 1787, 1720, 2081, 2149, 224, 2147, 2147, 2147, 233, 2147, 107, 2149, 452, 2149]
[13, 16, 61, 85, 94, 97, 109, 121, 189, 203, 233, 249, 265, 276, 284, 309, 359, 392, 393, 393, 393, 426, 441, 452, 452, 452, 561, 579, 629, 712, 739, 749, 749, 749, 792, 792, 792, 792, 809,

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

49
0.34265734265734266
